In [170]:
# それぞれのディレクトリの中でファイルを１つにまとめる
$ cat * > in.csv

SyntaxError: invalid syntax (<ipython-input-170-ac14382dbeae>, line 2)

In [183]:
!touch out.csv

In [184]:
#  名詞を取り出す関数
import re
import numpy as np
import pyprind
import pandas as pd
import os
from gensim import corpora
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

def createNone(in_name):
    words = ""
    with open("out.csv") as f:
        lines = f.readlines()
    with open("out.csv", "w") as f_out:
        for line in lines:
            f_out.write("{}\n".format(line[:-1]))
        with open("{}".format(in_name)) as f_in:        
            line = f_in.readline()
            row_line = re.match("[^,]*,([^,]*)", line)
            row = row_line.group(1)
            while line:
                search_line = re.match("([^,]*,){3}([^,]*),([^,]*)", line)
                if search_line is not None:
                    if search_line.group(3) == "名詞":
                        words += " " + search_line.group(2)
                line = f_in.readline()
                row_line = re.match("[^,]*,([^,]*)", line)
                if row_line is None:
                    f_out.write("{}\n".format(words))
                    break
#    print(words)
    

In [185]:
# それぞれの名詞を取り出す
createNone("aiduyaiti/in.csv")
createNone("akita/in.csv")

In [186]:
# 単語を特徴ベクトルに変換
count = CountVectorizer()
with open("out.csv") as f:
    lines = f.readlines()
    bag = count.fit_transform(lines)
    
print(count.vocabulary_)

{'人間': 486, 'やう': 244, 'つて': 163, 'こと': 88, '根気': 1335, 'それ': 136, 'やつ': 245, '城壁': 801, '祭壇': 1647, '神像': 1643, '殿堂': 1396, 'いつ': 32, 'この世': 89, 'ため': 150, '東洋': 1322, '西洋': 1848, '何処': 526, 'まま': 227, 'びた': 211, '腐肉': 1762, '素質': 1682, '遺族': 1988, '友人': 692, 'もの': 241, 'せつ': 125, 'かく': 60, '粉々': 1671, '地中': 791, '四角': 774, '御参り': 1055, '故人': 1202, '貞女': 1905, '孝子': 889, '貧乏': 1908, 'こんな': 93, '孝行': 890, 'むづかしい': 236, '建物': 1023, '手わざ': 1159, '時代': 1265, 'ところ': 171, '一つ': 367, '風霜': 2092, '曝露': 1281, '平気': 1003, '野火': 2006, '火事': 1485, '崩壊': 986, '立派': 1666, '石碑': 1630, '世間': 444, 'うち': 42, '土台': 783, '石垣': 1628, '下積み': 428, '追慕': 1959, '研究': 1633, 'たち': 145, '搨拓': 1193, '磨滅': 1638, '村里': 1318, '那全': 1990, '山河': 981, '記念': 1874, '無く': 1492, 'いくら': 26, 'むかし': 233, '学識': 897, '手腕': 1165, '情味': 1113, '大官': 847, '政府': 1201, '懐柔': 1143, '平素': 1005, '山水': 980, '眺め': 1618, '好き': 875, '在任': 788, '同行': 734, '宇宙': 898, '開闢': 2030, '初め': 634, 'ぶん': 215, '遊び': 1975, 'わけ': 255, 'みんな': 231, '湮滅': 1

In [187]:
print(bag.toarray())

[[0 0 0 ... 0 0 1]
 [6 1 1 ... 1 1 0]]


In [188]:
import pyprind
import pandas as pd
import os

# 単語の関連性を評価
np.set_printoptions(precision=2)
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)

data_train = tfidf.fit_transform(count.fit_transform(lines)).toarray()
print(data_train)

[[0.   0.   0.   ... 0.   0.   0.01]
 [0.02 0.   0.   ... 0.   0.   0.  ]]


In [189]:
# 学習

from sklearn.ensemble import RandomForestClassifier

label_train = [0, 1]

estimator = RandomForestClassifier()

estimator.fit(data_train, label_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [190]:
# 元データを入れて試してみる

label_predict = estimator.predict(data_train)

print(label_predict)

[0 1]


In [191]:
print(estimator.score(data_train, label_train))

1.0


In [193]:
from sklearn.model_selection import train_test_split

# 6割を学習用、 4割を試験用にする
data_train_s, data_test_s, label_train_s, label_test_s = train_test_split(data_train, label_train, test_size=0.4)

# 学習用に切り出したやつだけで学習
estimator.fit(data_train_s, label_train_s)

# 予測。正解が分かってる場合は、predict関数じゃなくてこうやると、正解率出してくれる
print(estimator.score(data_test_s, label_test_s))

0.0


In [194]:
print(data_train)

[[0.   0.   0.   ... 0.   0.   0.01]
 [0.02 0.   0.   ... 0.   0.   0.  ]]
